# Simulating orbits with REBOUND

https://rebound.readthedocs.io/en/latest/api/


In [ ]:
pip install rebound

In [ ]:
%matplotlib inline
%matplotlib widget
import matplotlib.pyplot as plt
import numpy as np
import rebound
from IPython.display import display, clear_output

To run an N-body simulation, we need to create a simulation object first. We will also choose units that are convenient for astronomy.

In [ ]:
sim = rebound.Simulation()
sim.units = ('AU','yr','Msun')
sim.G = 39.476926408897626

Then, we add bodies to the simulation. We will star with the star (1.5 solar masses) add then planets in the system from the inside out. Six parameters are required to uniquely specify the orbit of each planet of mass m, and we will use the following observationally determined ones:
 - semi-major axis (a) in au
 - eccentricity (e)
 - inclination (i) in radians
 - PA of the ascending node (Ω) in radians
 - argument of periastron (ω) in radians
 - mean anomaly at a reference epoch (MA_ref) in radians
 

In [ ]:
# Central star HR 8799 A
#sim.add(m=1e-6)                
sim.add(m=1.50)                

# co-planar orbits
dtor = np.pi / 180
inc_pl = 27.8 * dtor
Omega_pl = 60.1 * dtor

# planet masses
mass_e = 5e-3
mass_d = mass_e

# add inner most planet: HR 8799 e
sim.add(m=mass_e, a=16.64, e=0.1397, omega=118.8*dtor, M=1.261, inc=inc_pl, Omega=Omega_pl)
# add next planet: HR 8799 d
sim.add(m=mass_d, a=26.29, e=0.1112, omega=24.3*dtor, M=2.032, inc=inc_pl, Omega=Omega_pl)

# compute and print orbital elements of each planet
print()
print("      a (au)          e       omega (deg)  inc (deg)    Omega (deg)")
for o in sim.calculate_orbits(): 
    print("%12.4f %12.4f %12.4f %12.4f %12.4f " % (o.a, o.e, o.omega/dtor, o.inc/dtor, o.Omega/dtor))

Now let's set the time steps for numerical integration. A general rule of thumb is that time steps should be at most 10% of the shortest orbital period. The following will compute all orbital elements for our simlated particles and print out their periods. We then select an appropriate time step.

In [ ]:
# compute and print orbital period of each planet
print("   P (yr) ")
for o in sim.calculate_orbits(): 
    print("%9.3f" % o.P)

# set the time step
sim.dt = 1.0

We want to work in the center-of-mass frame throughout this simulation. Now that we have set up some planets we can do that. Let's look at the positions of the host star and two planets before integrating our simulation forward. Are their distances from the origin about where we expect them?

In [ ]:
# move to center of mass frame
sim.move_to_com()

# examine initial locations of particles
print("   X (au)    Y (au)    Z (au) ")
for p in sim.particles:
    print("%9.3f %9.3f %9.3f" % (p.x,p.y,p.z))

For fun, let's plot what the orbits we've specified look like and where our planets are to start out with. We'll also save this first timestep so that we can return to it later.

In [ ]:
# plot our starting point
op = rebound.OrbitPlot(sim, color=True, periastron=False, unitlabel="[au]",xlim=[-32,32.],ylim=[-32,32.])

# save our starting point
sim.save("start.bin")

Now let's actually integrate the orbits forward. The code below will iteratively integrate more and more time, stopping occasionally to display where the planets are at. Feel free to play with the number of frames and the time between displaying frames

Note: once you've run this block once, the simulation will always pick up where you left off. It won't start from the initial conditions. This is good if you want to keep exploring farther and farther in time, but bad if you want to re-run the experiment to check numbers.

In [ ]:
nframe = 500        # at home, I found that 500 frames
framestep = sim.dt  # at 1 yr/frame took ~30 s to render

fig = op.fig
for i in range(nframe):
    op.sim.integrate(sim.t + framestep) # timestep the sim
    clear_output(wait=True)             # clean up old plot
    op.update()                         # update plot data
    fig.canvas.draw()                   # redraw figure

Let's see if the orbital elements of the planets have changed. In a stable system, they should only change negligibly with time.

In [ ]:
# check time run so far
sim.status(showAllFields=False)

# move to center of mass frame
sim.move_to_com()

# compute and print orbital elements of each planet
print()
print("      a (au)          e       omega (deg)  inc (deg)    Omega (deg)")
for o in sim.calculate_orbits(): 
    print("%12.4f %12.4f %12.4f %12.4f %12.4f " % (o.a, o.e, o.omega/dtor, o.inc/dtor, o.Omega/dtor))

There are actually two more (known) planets in the HR 8799 system. So let's go ahead and return to our original time step and add those in.

In [ ]:
# reset sim
del sim
sim = rebound.Simulation("start.bin")

# planet masses
mass_c = mass_e
mass_b = 3e-3

sim.add(m=mass_c, a=43.12, e=0.0561, omega=28.5*dtor, M=4.232, inc=inc_pl, Omega=Omega_pl)
sim.add(m=mass_b, a=70.50, e=0.0113, omega=213.6*dtor, M=2.642, inc=inc_pl, Omega=Omega_pl)

# move to center of mass frame
sim.move_to_com()

# verify the reset
sim.status(showAllFields=False)

# compute and print orbital elements of each planet
print()
print("      a (au)          e       omega (deg)  inc (deg)    Omega (deg)")
for o in sim.calculate_orbits(): 
    print("%12.4f %12.4f %12.4f %12.4f %12.4f " % (o.a, o.e, o.omega/dtor, o.inc/dtor, o.Omega/dtor))

In [ ]:
op = rebound.OrbitPlot(sim, color=True, periastron=False, unitlabel="[au]")

Now let's run a long integration and then see how it looks at the end. (You can try running for as long as you like!)

In [ ]:
# run simulation
sim.integrate(100000)

# compute and print orbital elements of each planet
print()
print("      a (au)          e       omega (deg)  inc (deg)    Omega (deg)")
for o in sim.calculate_orbits(): 
    print("%12.4f %12.4f %12.4f %12.4f %12.4f " % (o.a, o.e, o.omega/dtor, o.inc/dtor, o.Omega/dtor))
    
# sim.status(showAllFields=False)

Now let's consider the motion of the star with respect to the barycentre by zooming in on it.

In [ ]:
op = rebound.OrbitPlot(sim,color=True,unitlabel="[AU]",xlim=[-1,1],ylim=[-1,1],orbit_style="solid")

In [ ]:
nframe = 500
framestep = 1*sim.dt

fig = op.fig
for i in range(nframe):
    op.sim.integrate(sim.t + framestep) # timestep the sim
    #clear_output(wait=True)             # clean up old plot
    op.update()                         # update plot data
    fig.canvas.draw()                   # redraw figure

Finally, let's completely reimagine the system by assigning them the same orbital parameters but higher masses. Play with increasing the masses until you start to see dynamical instabilities show up in the animation. Based on their luminosities, we expect e, d, and c to all be about the same mass, and b is expected to be a bit lower.

In [ ]:
# reset sim
del sim
sim = rebound.Simulation()

# Central star HR 8799 A
sim.add(m=1.50)                

# planet masses
mass_e = 7.3e-3
mass_d = mass_e
mass_c = mass_e
mass_b = mass_e * 0.5

sim.add(m=mass_e, a=16.64, e=0.1397, omega=118.8*dtor, M=1.261, inc=inc_pl, Omega=Omega_pl)
sim.add(m=mass_d, a=26.29, e=0.1112, omega=24.3*dtor, M=2.032, inc=inc_pl, Omega=Omega_pl)
sim.add(m=mass_c, a=43.12, e=0.0561, omega=28.5*dtor, M=4.232, inc=inc_pl, Omega=Omega_pl)
sim.add(m=mass_b, a=70.50, e=0.0113, omega=213.6*dtor, M=2.642, inc=inc_pl, Omega=Omega_pl)

sim.move_to_com()

After setting the planet masses, run the simulation for a large number of steps. Note that if the system disintegrates early in the time specified, it will slow down the simulation greatly. So if it seems to taking forever, then start over with either lower masses or shorter time.

In [ ]:
# run simulation
sim.integrate(1000000)

# plot endpoint
op = rebound.OrbitPlot(sim, color=True, periastron=False, unitlabel="[au]")

In [ ]:
# use this for reporting the final state of your simulation
sim.status(showAllFields=False)